# Download do RegulonDB

* RegulonDB (https://regulondb.ccg.unam.mx)


In [1]:
# Criar o diretorio

!pwd
!mkdir -pv ./data/RegulonDB/raw

/home/alisson/work/github_chiquittobioinfo_srna
mkdir: created directory './data/RegulonDB'
mkdir: created directory './data/RegulonDB/raw'


In [2]:
# Download dos arquivos

# Acessar o site > Downloads > Experimental Datasets

# E. coli K-12 genome sequence raw format
![ ! -f ./data/RegulonDB/raw/E_coli_K12_MG1655_U00096.3.txt ] && \
  wget --no-check-certificate https://regulondb.ccg.unam.mx/menu/download/datasets/files/E_coli_K12_MG1655_U00096.3.txt -P ./data/RegulonDB/raw

# Gene Sequence
![ ! -f ./data/RegulonDB/raw/Gene_sequence.txt ] && \
  wget --no-check-certificate https://regulondb.ccg.unam.mx/menu/download/datasets/files/Gene_sequence.txt -P ./data/RegulonDB/raw

# Gene - Product > sRNA genes
![ ! -f ./data/RegulonDB/raw/sRNASet.txt ] && \
  wget --no-check-certificate https://regulondb.ccg.unam.mx/menu/download/datasets/files/sRNASet.txt -P ./data/RegulonDB/raw

--2023-03-10 17:46:57--  https://regulondb.ccg.unam.mx/menu/download/datasets/files/E_coli_K12_MG1655_U00096.3.txt
Resolving regulondb.ccg.unam.mx (regulondb.ccg.unam.mx)... 132.248.34.160
Connecting to regulondb.ccg.unam.mx (regulondb.ccg.unam.mx)|132.248.34.160|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 4703569 (4,5M) [text/plain]
Saving to: ‘./data/RegulonDB/raw/E_coli_K12_MG1655_U00096.3.txt’

E_coli_K12_MG1655_U 100%[===================>]   4,49M   222KB/s    in 30s     

2023-03-10 17:47:28 (155 KB/s) - ‘./data/RegulonDB/raw/E_coli_K12_MG1655_U00096.3.txt’ saved [4703569/4703569]

--2023-03-10 17:47:29--  https://regulondb.ccg.unam.mx/menu/download/datasets/files/Gene_sequence.txt
Resolving regulondb.ccg.unam.mx (regulondb.ccg.unam.mx)... 132.248.34.160
Connecting to regulondb.ccg.unam.mx (regulondb.ccg.unam.mx)|132.248.34.160|:443... connected.
  Unable to locally verify the issuer's authority.
HTT

In [3]:
# Compactar para commit no git
![ ! -f ./data/RegulonDB/raw/E_coli_K12_MG1655_U00096.3.txt.xz ] && \
  xz -czk9ve -T 0 ./data/RegulonDB/raw/E_coli_K12_MG1655_U00096.3.txt > ./data/RegulonDB/raw/E_coli_K12_MG1655_U00096.3.txt.xz

![ ! -f ./data/RegulonDB/raw/Gene_sequence.txt.xz ] && \
  xz -czk9ve -T 0 ./data/RegulonDB/raw/Gene_sequence.txt > ./data/RegulonDB/raw/Gene_sequence.txt.xz

![ ! -f ./data/RegulonDB/raw/sRNASet.txt.xz ] && \
  xz -czk9ve -T 0 ./data/RegulonDB/raw/sRNASet.txt > ./data/RegulonDB/raw/sRNASet.txt.xz

./data/RegulonDB/raw/E_coli_K12_MG1655_U00096.3.txt (1/1)
  100 %   1.227,4 KiB / 4.593,3 KiB = 0,267   1,2 MiB/s       0:03             
./data/RegulonDB/raw/Gene_sequence.txt (1/1)
  100 %   1.159,3 KiB / 4.615,0 KiB = 0,251   1,3 MiB/s       0:03             
./data/RegulonDB/raw/sRNASet.txt (1/1)
  100 %          5.012 B / 18,1 KiB = 0,271                                    


In [4]:
!ls -lah ./data/RegulonDB/raw

total 12M
drwxrwxr-x 2 alisson alisson 4,0K mar 10 17:47 .
drwxrwxr-x 3 alisson alisson 4,0K mar 10 17:46 ..
-rw-rw-r-- 1 alisson alisson 4,5M jun 10  2020 E_coli_K12_MG1655_U00096.3.txt
-rw-rw-r-- 1 alisson alisson 1,2M mar 10 17:47 E_coli_K12_MG1655_U00096.3.txt.xz
-rw-rw-r-- 1 alisson alisson 4,6M dez 13 13:54 Gene_sequence.txt
-rw-rw-r-- 1 alisson alisson 1,2M mar 10 17:47 Gene_sequence.txt.xz
-rw-rw-r-- 1 alisson alisson  19K dez 13 13:54 sRNASet.txt
-rw-rw-r-- 1 alisson alisson 4,9K mar 10 17:47 sRNASet.txt.xz


# Obter as sequências

Será utilizado o bedtools para obter as sequências. Para isso vamos precisar criar um arquivo com as coordenadas no formato .bed.

A BED file (.bed) is a tab-delimited text file that defines a feature track. The BED file format is described on the UCSC Genome Bioinformatics web site: <http://genome.ucsc.edu/FAQ/FAQformat>.  
Ref: <https://software.broadinstitute.org/software/igv/BED>

Um arquivo BED tem as colunas `chrom`, `chromStart`, `chromEnd`, `name`, `score` e `strand` delimitadas por tab.

In [5]:
# O arquivo de entrada possui um cabecalho com instrucoes
# O tail 43 foi usado para ignorar as primeiras 43 linhas (cabecalho)
# Foi gerado o bed com um -1 no chromStart para corresponder com os outros arquivos do dataset (ex.: Gene_Sequence.csv)

!tail -n +43 ./data/RegulonDB/raw/sRNASet.txt | awk 'BEGIN { FS="\t"; OFS="\t" } { if($6=="forward") { print "E_coli_K12_MG1655_U00096.3",$4-1,$5,$1,1,"+" } }' > ./data/RegulonDB/sRNASet.bed
!tail -n +43 ./data/RegulonDB/raw/sRNASet.txt | awk 'BEGIN { FS="\t"; OFS="\t" } { if($6=="reverse") { print "E_coli_K12_MG1655_U00096.3",$4-1,$5,$1,1,"-" } }' >> ./data/RegulonDB/sRNASet.bed
!wc -l ./data/RegulonDB/sRNASet.bed

111 ./data/RegulonDB/sRNASet.bed


In [6]:
# Obter as sequencias
!rm ./data/RegulonDB/raw/E_coli_K12_MG1655_U00096.3.txt.fai
!bedtools getfasta -s -fi ./data/RegulonDB/raw/E_coli_K12_MG1655_U00096.3.txt -bed ./data/RegulonDB/sRNASet.bed -name -bedOut > ./data/RegulonDB/sRNASet.tsv

rm: cannot remove './data/RegulonDB/raw/E_coli_K12_MG1655_U00096.3.txt.fai': No such file or directory
index file ./data/RegulonDB/raw/E_coli_K12_MG1655_U00096.3.txt.fai not found, generating...


In [7]:
# Converter TSV para FASTA
!awk '{print ">" $4 "\n" $7}' ./data/RegulonDB/sRNASet.tsv > ./data/RegulonDB/sRNASet.fa

In [8]:
# Remover arquivos temporarios
!rm -v ./data/RegulonDB/sRNASet.{bed,tsv}

removed './data/RegulonDB/sRNASet.bed'
removed './data/RegulonDB/sRNASet.tsv'
